### Data analysys
This notebook conducts exploratory data analysis to support a deeper understanding of the dataset.

In [7]:
# for the xlsx
%pip install pandas openpyxl
%pip install opencv-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.5 MB ? eta -:--:--
   - -------------------------------------- 1.0/39.5 MB 2.8 MB/s eta 0:00:14
   - -------------------------------------- 1.6/39.5 MB 2.9 MB/s eta 0:00:14
   -- ------------------------------------- 2.1/39.5 MB 2.7 MB/s eta 0:00:14
   -- ------------------------------------- 2.6/39.5 MB 2.6 MB/s eta 0:00:15
   --- ------------------------------------ 3.1/39.5 MB 2.6 MB/s eta 0:00:15
   --- ------------------------------------ 3.7/39.5 MB 2.6 MB/s eta 0:00:14
   ---- ----------------------------------- 4.2/39.5 MB 2.5 MB/s eta 0:00:14
   ---- ----------------------------------- 4.5/39.5 MB 2.5 MB/s eta 0:00:14
   ----- ---------------------------------- 5.0/39.5 MB 2.4 MB/s eta 0:00:15
   ----- ---------------------------------- 5.2/39.5 MB 2.3 MB/s eta 0:00:15
   ----- ---------------------------------- 5.8/39.5 MB 2.3 MB/s eta 0:00:15
   ------ --


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import pandas as pd
import cv2
import os

In [9]:
PATH_OSATS = "../data/raw/OSATS.xlsx"
PATH_DATA_RAW = "../data/raw/"
PATH_DATA_PROCESSED = "../data/processed/"

In [4]:
osats = pd.read_excel(PATH_OSATS, engine="openpyxl")
osats


,STUDENT,GROUP,TIME,SUTURES,INVESTIGATOR,VIDEO,OSATS_RESPECT,OSATS_MOTION,OSATS_INSTRUMENT,OSATS_SUTURE,OSATS_FLOW,OSATS_KNOWLEDGE,OSATS_PERFORMANCE,OSATS_FINAL_QUALITY,GLOBA_RATING_SCORE
0,AHO729,E-LEARNING,PRE,1.0,A,P54M,2,1,2,2,1,2,2,1,13
1,AHO729,E-LEARNING,PRE,1.0,B,P54M,2,1,3,1,2,1,2,1,13
2,AHO729,E-LEARNING,PRE,1.0,C,P54M,2,1,1,1,1,1,1,1,9
3,AHO729,E-LEARNING,POST,4.5,A,M45P,4,4,4,3,3,4,3,3,28
4,AHO729,E-LEARNING,POST,4.5,B,M45P,2,3,4,3,3,4,3,3,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937,ZWO108,HMD-BASED,PRE,0.5,B,D94Z,1,1,2,1,1,1,1,1,9
938,ZWO108,HMD-BASED,PRE,0.5,C,D94Z,1,1,1,1,2,1,1,1,9
939,ZWO108,HMD-BASED,POST,3.0,A,Z49D,4,3,3,3,3,3,3,3,25
940,ZWO108,HMD-BASED,POST,3.0,B,Z49D,4,3,4,4,4,5,4,5,33


In [5]:
def grs_to_class(grs):
    if 8 <= grs <= 15:
        return 0  # Classe 0 (novato)
    elif 16 <= grs <= 23:
        return 1  # Classe 1 (intermédio)
    elif 24 <= grs <= 31:
        return 2  # Classe 2 (proficiente)
    elif 32 <= grs <= 40:
        return 3  # Classe 3 (especialista)
    else:
        return -1  # Caso o valor de GRS seja inválido (fora do esperado)

osats['GRS'] = osats['GLOBA_RATING_SCORE'].apply(grs_to_class)

osats.head()

,STUDENT,GROUP,TIME,SUTURES,INVESTIGATOR,VIDEO,OSATS_RESPECT,OSATS_MOTION,OSATS_INSTRUMENT,OSATS_SUTURE,OSATS_FLOW,OSATS_KNOWLEDGE,OSATS_PERFORMANCE,OSATS_FINAL_QUALITY,GLOBA_RATING_SCORE,GRS
0,AHO729,E-LEARNING,PRE,1.0,A,P54M,2,1,2,2,1,2,2,1,13,0
1,AHO729,E-LEARNING,PRE,1.0,B,P54M,2,1,3,1,2,1,2,1,13,0
2,AHO729,E-LEARNING,PRE,1.0,C,P54M,2,1,1,1,1,1,1,1,9,0
3,AHO729,E-LEARNING,POST,4.5,A,M45P,4,4,4,3,3,4,3,3,28,2
4,AHO729,E-LEARNING,POST,4.5,B,M45P,2,3,4,3,3,4,3,3,25,2


Frame reduction 


In [20]:
def reduce_frames(video_path, output_dir, target_fps=5):

    video = cv2.VideoCapture(video_path)

    video_name = os.path.splitext(os.path.basename(video_path))[0]
    video_output_folder = os.path.join(output_dir, video_name)

    if not os.path.exists(video_output_folder):
        os.makedirs(video_output_folder)

    fps = video.get(cv2.CAP_PROP_FPS)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    
    interval = int(fps / target_fps)

    for frame_num in range(total_frames):
        ret, frame = video.read()
        
        if ret and frame_num % interval == 0:
            frame_name = os.path.join(video_output_folder, f"frame_{frame_num}.jpg")
            cv2.imwrite(frame_name, frame)

    video.release()

    print(f"Frames saved to {video_name}")


In [23]:
def process_all_videos_in_folder(input_folder, output_folder, package, target_fps=5):

    package_folder = os.path.join(output_folder, package)
    if not os.path.exists(package_folder):
        os.makedirs(package_folder)

    video_folder = os.path.join(input_folder, package)
    
    for filename in os.listdir(video_folder):
        if filename.endswith(('.mp4')):
            video_path = os.path.join(video_folder, filename)

            reduce_frames(video_path, package_folder, target_fps)


Reduction package to package

In [ ]:
#process_all_videos_in_folder(PATH_DATA_RAW , PATH_DATA_PROCESSED ,'Package 02', target_fps=5)
#rocess_all_videos_in_folder(PATH_DATA_RAW , PATH_DATA_PROCESSED ,'Package 03', target_fps=5)

In [24]:
process_all_videos_in_folder(PATH_DATA_RAW , PATH_DATA_PROCESSED ,'Package 01', target_fps=5)

Frames saved to A31H
Frames saved to A36O
Frames saved to A40E
Frames saved to A41X
Frames saved to A59Z
Frames saved to A61V
Frames saved to A66S
Frames saved to A80U
Frames saved to A83X
Frames saved to A88O
Frames saved to A92O
Frames saved to A99X
Frames saved to A99Y
Frames saved to B12G
Frames saved to B36R
Frames saved to B53D
Frames saved to B61D
Frames saved to B68G
Frames saved to B88L
Frames saved to C16G
Frames saved to C18X
Frames saved to C28D
Frames saved to C33I
Frames saved to C36X
Frames saved to C42I
Frames saved to C43T
Frames saved to C46R
Frames saved to C48S
Frames saved to C56Y
Frames saved to C58C
